Due to the lack of GPU on my local machine, I used Google Colab for this part of the code as it involves training the model with a ResNet-50 architecture.

I then saved the model and tested it with real time data on my local machine. The code for this is in Mask-Detection-#2

In [ ]:
import torch
print(torch.__version__)

1.5.0+cu101


In [ ]:
pip install xmltodict

In [ ]:
import os
import cv2
import xmltodict
import numpy as np
import torch 
import torchvision.transforms as tt
import torchvision.datasets 
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt



In [ ]:
#Mounting google drive to colab.

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My Drive/Kaggle

# Dataset Collection and Organization

In [ ]:
#Adding all images to the list - 'images'

images = []
for dirname, _, filenames in os.walk('/content/gdrive/My Drive/Kaggle'):
  for file_path in filenames:
    extension = file_path[len(file_path) - 4:]
    if extension != '.xml':
      images.append(file_path)
del images[0:1]



In [ ]:
#For the given image name, this function returns the path for that particular image and it's label.

def get_path(image_name):

    home_dir = '/content/gdrive/My Drive/Kaggle/'
    image_path = home_dir + 'images/' + image_name
    
    if image_name[-4:] == 'jpeg':
        label_name = image_name[:-5] + '.xml'
    else:
        label_name = image_name[:-4] + '.xml'
    
    label_path = home_dir + 'labels/' + label_name
    
    return  image_path, label_path

In [ ]:
#Returns the coordinates and size of the bounding box around the faces.

def parse_xml(label_path):
    x = xmltodict.parse(open(label_path , 'rb'))
    item_list = x['annotation']['object']
    
    # when image has only one face
    if not isinstance(item_list, list):
        item_list = [item_list]
        
    result = []
    #when image has multiple faces
    for item in item_list:
        name = item['name']
        bndbox = [(int(item['bndbox']['xmin']), int(item['bndbox']['ymin'])),
                  (int(item['bndbox']['xmax']), int(item['bndbox']['ymax']))]       
        result.append((name, bndbox))
    
    size = [int(x['annotation']['size']['width']), 
            int(x['annotation']['size']['height'])]
    
    return result, size



In [ ]:
#Crops the faces in the image using the bounding box coordinates and adds each one to a list with the corresponding label.

def get_image(image_name, bndbox=True):

    new_image = []

    image_path, label_path = get_path(image_name)
    bgr_image = cv2.imread(image_path)

    if bndbox:
        labels, size = parse_xml(label_path)
        
        for label in labels:
            name, bndbox = label

            small_image = bgr_image[bndbox[0][1]:bndbox[1][1], bndbox[0][0]:bndbox[1][0]]
            
            if name == 'good':
                label_num = 0
            elif name == 'bad':
                label_num = 1
            else:
                label_num = 2

            n_image = [small_image, label_num]

            new_image.append(n_image)
    
    return new_image

In [ ]:
#Creates a separate directory for images of each label and adds each image to the corresponding folder.

def make_directory(dir_name):
  try:
      os.mkdir(dir_name)
  except FileExistsError:
    print("directory" + dir_name + "already exists")

main_dir = "train/"
label_0_dir = main_dir + "0/"
label_1_dir = main_dir + "1/"
label_2_dir = main_dir + "2/"

make_directory(main_dir)
make_directory(label_0_dir)
make_directory(label_1_dir)
make_directory(label_2_dir)

label_0_count = 0
label_1_count = 0
label_2_count = 0

for x in images:
  print(x)
  up_image = get_image(x)


  for im in up_image:
    label = im[1]
    image = im[0]

    if label == 0:
      cropped_image = str(label_0_count) + '.jpg'
      cv2.imwrite(label_0_dir + cropped_image, image)
      label_0_count += 1

    elif label == 1:
      cropped_image = str(label_1_count) + '.jpg'
      cv2.imwrite(label_1_dir + cropped_image, image)
      label_1_count += 1

    elif label == 2:
      cropped_image = str(label_2_count) + '.jpg'
      cv2.imwrite(label_2_dir + cropped_image, image)
      label_2_count += 1







In [ ]:
filenames_label_0 = [f for f in os.listdir(label_0_dir) if os.path.isfile(os.path.join(label_0_dir, f))] 
filenames_label_1 = [f for f in os.listdir(label_1_dir) if os.path.isfile(os.path.join(label_1_dir, f))] 
filenames_label_2 = [f for f in os.listdir(label_2_dir) if os.path.isfile(os.path.join(label_2_dir, f))]



In [ ]:
print(len(filenames_label_0) + len(filenames_label_1) + len(filenames_label_2) )

3914


## **TRAINING THE MODEL**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model = torchvision.models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [ ]:
model.fc = nn.Sequential(nn.Linear(2048, 1000),
                         nn.ReLU(),
                         nn.Dropout(),
                         nn.Linear(1000, 3))



In [ ]:
model.to(device)

In [ ]:
stats= ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
train_tfms = tt.Compose([tt.Resize((224, 224)),
                        tt.ToTensor(),
                        tt.Normalize(*stats)])

In [ ]:
dataset = ImageFolder(main_dir, transform=train_tfms)

In [ ]:
dataset_size = len(dataset)
val_size = int(dataset_size * 0.2)
train_size = dataset_size - val_size

train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])


In [ ]:
BATCH_SIZE = 45

train_loader = torch.utils.data.DataLoader(train_set,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False,
                                          pin_memory=True)
                

val_loader = torch.utils.data.DataLoader(val_set,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False,
                                          pin_memory=True)

In [ ]:
total_epoch = 20
learning_rate = 0.003
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 



In [ ]:
make_directory("models")
models_dir = "models/"

directorymodelsalready exists


In [ ]:
#For a given number of epochs, it trains the model and checks accuracy against the validation set.

i=0
train_loss=[]
val_loss=[]
for epoch in range(total_epoch):
    
    torch.manual_seed(2 + epoch)
    epoch_train_loss = 0
    count=0
    for X, y in train_loader:
        
        count += 1
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        result = model(X)
        loss = loss_fn(result, y)
        epoch_train_loss += loss.item()
        loss.backward()
        optimizer.step()
      
    train_loss.append(epoch_train_loss)
    
    
    epoch_val_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for X, y in val_loader:
          
            X, y = X.to(device), y.to(device)
             
            result = model(X)
            loss = loss_fn(result, y)
            epoch_val_loss += loss.item()
            _, maximum = torch.max(result.data, 1)
            total += y.size(0)
            correct += (maximum == y).sum().item()
            
        val_loss.append(epoch_val_loss)
    accuracy = correct/total

    print("Epoch:", epoch, ", Training Loss:", epoch_train_loss, ", Validation Loss:", epoch_val_loss, ", Accuracy: ", accuracy)

    #Finds the epoch with the least validation loss and saves the model

    if min(val_loss)==val_loss[-1]:
      best_epoch = epoch

      checkpoint = {"model":model,
                  "state_dict":model.state_dict(),
                  "optimizer":optimizer.state_dict()}

      torch.save(checkpoint, models_dir + '{}.pth'.format(epoch))

In [ ]:
#Loads the specified model that was saved earlier

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    return model.eval()

#filepath = models_dir + "5.pth"
filepath = models_dir + str(best_epoc) + ".pth"
loaded_model = load_checkpoint(filepath)

In [ ]:
img, label = dataset[0]
xb = img.unsqueeze(0)
xb = xb.cuda()
yb = model(xb)
print(yb)

tensor([[-0.0349,  0.0049,  0.1385]], device='cuda:0', grad_fn=<AddmmBackward>)


In [ ]:
def predict_image(img, model):
    # Convert to a batch of 1
    xb = img.unsqueeze(0)
    xb = xb.to(device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    pre = preds.item()
    print(pre)
    # Retrieve the class label
    return dataset.classes[preds[0].item()]

In [ ]:
img, label = dataset[3456]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, loaded_model))